In [80]:
import pandas as pd
import qiime2 as q2
import biom

## Genome Coverage Filter

In [85]:
md = pd.read_csv('../data/64306_64306_analysis_mapping.txt', sep = '\t', dtype={'#SampleID': str}).set_index('#SampleID')

/tmp/ipykernel_245036/299696563.py:1: DtypeWarning: Columns (311,492,532,866) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('../data/64306_64306_analysis_mapping.txt', sep = '\t', dtype={'#SampleID': str}).set_index('#SampleID')


In [86]:
#Feature table from Qiita analysis 64306
qiita_table = biom.load_table("/home/lakhatib/thdmi/data/64306_analysis_Metagenomic_Woltkav014DatabasescratchqpwoltkaWoLr2WoLr2BIOMnonebiom.biom").to_dataframe().transpose()
#import coverages from Micov
cov = pd.read_csv('coverages.coverage.tsv', sep = '\t')

In [87]:
ft = qiita_table.transpose()
cov = cov.set_index('genome_id')

cov = cov.loc[cov.index.intersection(ft.index)]

ft['percent_covered'] = cov['percent_covered']

#check how many features will remain with a 15% threshold 
sum(ft['percent_covered'] > 25)

1337

In [88]:
#apply filter
ft_filtered = ft[ft['percent_covered']  > 25]
ft_filtered = ft_filtered.transpose()

ft_filtered = ft_filtered.drop("percent_covered")

## Sample Filtering

In [89]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 2747
Mexico            549
US                534
Japan             524
Spain             481
UK                366
not applicable    277
unknown            10
control sample      6
Name: thdmi_cohort, dtype: int64


In [90]:
#filter to valid countries
md = md.loc[md['thdmi_cohort'].isin(['Mexico', 'US', 'UK'])]

In [91]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1449
Mexico    549
US        534
UK        366
Name: thdmi_cohort, dtype: int64


In [92]:
md = md.loc[md.keep_sample_for_thdmi == 'yes']

In [93]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1291
Mexico    507
US        442
UK        342
Name: thdmi_cohort, dtype: int64


In [94]:
md = md.loc[md.index.intersection(ft_filtered.index)]

In [95]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1291
Mexico    507
US        442
UK        342
Name: thdmi_cohort, dtype: int64


In [96]:
#drop samples with invalid covariates
import numpy as np

md = md.loc[md['host_age'] != 'not provided']
md['host_age'] = md['host_age'].astype('float')
md = md.loc[(md['host_age'] >= 18) & (md['host_age'] <= 100)]

md['host_body_mass_index'] = md['host_body_mass_index'].replace('not provided', np.nan)

md['host_body_mass_index'] = md['host_body_mass_index'].astype(float)

md['host_body_mass_index'] = np.where((md['host_body_mass_index'] < 12) | (md['host_body_mass_index'] > 70), np.nan, 
                                            md['host_body_mass_index']) 

md['covid_level_of_wellbeing'] = md['covid_level_of_wellbeing'].replace('not provided', np.nan)

md['antibiotic_history'] = md['antibiotic_history'].replace('not provided', np.nan)

md = md.dropna(subset = ['antibiotic_history', 'covid_level_of_wellbeing', 'host_body_mass_index', 'host_age'])

In [97]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1218
Mexico    497
US        412
UK        309
Name: thdmi_cohort, dtype: int64


In [98]:
#match ft samples to metadata
ft_filtered = ft_filtered.loc[ft_filtered.index.intersection(md.index)]

## Feature Filtering

In [99]:
#save as .qza 

ft_q2 = q2.Artifact.import_data('FeatureTable[Frequency]', ft_filtered)
ft_q2.save('../data/filtered_feature-table.qza')

'../data/filtered_feature-table.qza'

In [100]:
# filter against gg2
!qiime greengenes2 filter-features --i-feature-table /home/lakhatib/3country/final_scripts/data/filtered_feature-table.qza --i-reference /home/mcdonadt/greengenes2/release/2022.10/2022.10.taxonomy.asv.nwk.qza --o-filtered-feature-table /home/lakhatib/3country/final_scripts/data/gg2_filtered-feature-table.qza

Saved FeatureTable[Frequency] to: /home/lakhatib/3country/final_scripts/data/gg2_filtered-feature-table.qza


In [101]:
#upload filtered table
ft_q2 = q2.Artifact.load('/home/lakhatib/3country/final_scripts/data/gg2_filtered-feature-table.qza')
ft = ft_q2.view(pd.DataFrame)

In [102]:
print(len(ft.columns))

944


In [103]:
# Define the threshold for relative abundance
relative_abundance_threshold = 0.000001  # 0.0001%

In [104]:
#Calculate total abundance for each sample
ft_ra = ft.copy()
ft_ra['total_abundance'] = ft.sum(axis=1)

In [105]:
ft_ra['total_abundance'] = ft_ra['total_abundance'] * relative_abundance_threshold

In [106]:
# Apply the threshold to each feature to check if it meets the relative abundance threshold
features_above_threshold = ft_ra.iloc[:, :-1].ge(ft_ra['total_abundance'], axis=0)

In [107]:
# Step 1: Calculate the percentage of samples where each feature meets the threshold
feature_prevalence = features_above_threshold.mean(axis=0)

In [108]:
# Step 2: Filter out features with prevalence < 10%
prevalence_threshold = 0.1  # 10%
features_to_keep = feature_prevalence[feature_prevalence >= prevalence_threshold].index

In [109]:
# Filter the original dataframe to keep only these features
filtered_features_df = ft_ra[features_to_keep]

In [110]:
filtered_features_df = filtered_features_df.loc[filtered_features_df.sum(axis=1) >= 1000000]

In [111]:
#make sure all samples are above 1000000
filtered_features_df.sum(axis=1).min()

1001132.0

In [112]:
md = md.loc[md.index.intersection(filtered_features_df.index)]

In [113]:
print(f"Total Samples: {len(md)}")
print(md['thdmi_cohort'].value_counts())

Total Samples: 1177
Mexico    460
US        410
UK        307
Name: thdmi_cohort, dtype: int64


In [114]:
md = md.rename_axis('#SampleID')

In [115]:
md.to_csv('../data/subsetted_md.tsv', sep = '\t')

In [116]:
filtered_features_df = filtered_features_df.loc[filtered_features_df.index.intersection(md.index)]

In [117]:
filtered_features_df.to_csv('../data/filtered_ft.tsv', sep = '\t')

In [118]:
filtered_features_q2 = q2.Artifact.import_data('FeatureTable[Frequency]', filtered_features_df)

In [119]:
filtered_features_q2.save('../data/filtered_ft.qza')

'../data/filtered_ft.qza'

In [120]:
filtered_features_df

,G000006605,G000006865,G000006925,G000007265,G000007325,G000007465,G000007525,G000007785,G000008345,G000008865,...,G902363595,G902363685,G902363725,G902363825,G902364485,G902373415,G902377395,G902385285,G902387315,G902406265
10317.X00178927,4.0,41.0,1112.0,7.0,1.0,0.0,36674.0,6.0,2.0,1253.0,...,208.0,1818.0,553.0,5290.0,475.0,6.0,22.0,42089.0,434.0,381.0
10317.X00178928,0.0,13.0,1038.0,6.0,3.0,2.0,135.0,77.0,4.0,1354.0,...,460.0,18745.0,1400.0,5420.0,3657.0,15.0,63.0,61441.0,2582.0,275.0
10317.X00178930,0.0,372.0,4.0,16.0,0.0,0.0,7311.0,14.0,11.0,5.0,...,71.0,1744.0,630.0,1799.0,253.0,2.0,19.0,11276.0,938.0,3.0
10317.X00178931,4.0,9.0,1.0,3.0,1.0,0.0,244.0,12.0,30.0,2.0,...,667.0,4113.0,377.0,2520.0,1066.0,394.0,36.0,31265.0,304.0,18.0
10317.X00178933,0.0,12.0,39.0,15.0,1.0,7.0,0.0,114.0,2.0,43.0,...,538.0,8746.0,903.0,1434.0,1850.0,11.0,59.0,53673.0,2465.0,196.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317.X00215921,11.0,11.0,1990.0,36.0,0.0,0.0,236.0,61.0,3.0,2192.0,...,490.0,1920.0,2188.0,1897.0,2055.0,51.0,41.0,3889.0,5794.0,768.0
10317.X00215925,30.0,30.0,2765.0,3.0,0.0,1.0,11361.0,5.0,5.0,3634.0,...,153.0,9596.0,602.0,1023.0,1705.0,3.0,31.0,17445.0,634.0,256.0
10317.X00215926,536.0,0.0,323.0,0.0,1.0,0.0,26.0,4.0,55.0,396.0,...,131.0,1451.0,119.0,1622.0,1591.0,71.0,46.0,2711.0,395.0,137.0
10317.X00215929,154.0,4333.0,4212.0,6.0,0.0,2.0,2638.0,68.0,7.0,4765.0,...,133.0,3472.0,147.0,1417.0,1084.0,43.0,35.0,11214.0,1566.0,274.0
